In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import MultiStepLR

import time

In [43]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [44]:
# CIFAR-100 데이터셋 불러오기
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [45]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            # conv1
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.SELU(inplace=True),
#             nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # conv2
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.SELU(inplace=True),
#             nn.BatchNorm2d(192),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # conv3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.SELU(inplace=True),
            # conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.SELU(inplace=True),
            # conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.SELU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.SELU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.SELU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [46]:
# 모델 인스턴스 생성
net = AlexNet()

# GPU를 사용할 수 있다면 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
# Optimizer 설정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# # 학습률 스케줄러 설정
# scheduler = MultiStepLR(optimizer, milestones=[60, 120], gamma=0.1)

In [47]:
total_params = sum(p.numel() for p in net.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 57413540


In [48]:
# 모델 학습
start_time = time.time()

for epoch in range(40):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 입력 받은 후
        inputs, labels = data[0].to(device), data[1].to(device)

        # gradient 매개변수 0으로 만든 후
        optimizer.zero_grad()

        # 순전파+역전파+최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 출력
        running_loss += loss.item()
        # if i % 2000 == 1999 조건문은 주로 일반적인 CIFAR-10 데이터셋에 대한 예시로 사용되었기
        # 때문에, CIFAR-100 데이터셋에 따라 출력 주기가 조정되어야 합니다.
        # 예를 들어, CIFAR-100 데이터셋에서는 if i % 200 == 199로 변경하여 200번째 배치마다 평균 손실을 출력하도록 설정할 수 있습니다.
        # 이렇게 조정하면 더 적은 배치 단위로 평균 손실을 확인할 수 있습니다.
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

        # # 학습률 스케줄링
        # scheduler.step()

end_time = time.time()
elapsed_time = end_time - start_time
print('Finished Training')
print('Elapsed Time: %.2f seconds' % elapsed_time)

[1,   200] loss: 4.402
[1,   400] loss: 4.126
[1,   600] loss: 3.965


KeyboardInterrupt: 

In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: %.2f %%' % (100 * correct / total))